Investigating the $L_{n,\boldsymbol y}$ matrix
==

In [ ]:
%matplotlib notebook
import mushi
import histories
import utils
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from numpy.linalg import cond, svd
from matplotlib.colors import LogNorm

### define a flat $\eta(t)$

In [ ]:
t = np.logspace(-1, 4, 100)

y = 100 * np.ones(len(t) + 1)
# y[:-1] += 9000 * np.exp(- .1 * t)
η = histories.η(t, y)

### The condition numbers of $L_{n,\mathbf y}$ for several values of $n$
Things go to 💩 for $n$ larger than a couple hundred or so. This is a problem. Most of the badness is from $C$.

In [ ]:
ns = np.logspace(1, 1.4, 100, dtype=np.int)
L = lambda n: utils.C(n) @ utils.M(n, η)
condition_numbers_L = [cond(L(n)) for n in ns]
condition_numbers_L_twid = [cond(L(n) / L(n).sum(1, keepdims=True)) for n in ns]

In [ ]:
plt.figure(figsize=(5, 3))
plt.plot(ns, condition_numbers_L)
plt.plot(ns, condition_numbers_L_twid)
plt.xlabel('$n$'); plt.ylabel('condition number')
plt.xscale('log'); plt.yscale('log')
plt.tight_layout()
plt.show()

### Singular value spectrum of $L_{n,\mathbf y}$ with $n=100$

In [ ]:
n = 100
sfs = mushi.kSFS(η, n=n)
u, s, vh = svd(sfs.L, full_matrices=False)
plt.figure(figsize=(6, 2))
plt.bar(np.arange(1, min(n, η.m + 1)), s)
plt.yscale('log')
plt.ylabel('singular value')    
plt.show()

### Top 10 right singular vectors

In [ ]:
df = pd.DataFrame(vh.T, index=pd.Index(np.concatenate((np.array([0]), t)), name='time'))
df = df.melt().set_index('variable')
df['singular value'] = s[df.index]
df['time'] = np.tile(np.concatenate((np.array([0]), t)), df.index[-1] + 1)

# filter to top 10 singular values
df = df[df.index < 10]

In [ ]:
plt.figure(figsize=(7, 2))
ax = sns.lineplot(x='time', y='value', hue='singular value', units='singular value', data=df, estimator=None,
#              palette=sns.color_palette("RdBu", n_colors=df['singular value'].nunique())
            )
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.tight_layout()
plt.xscale('log')
plt.show()

### TMRCA CDF

In [ ]:
plt.figure(figsize=(3, 2))
plt.plot(η.change_points, sfs.tmrca_cdf())
plt.xlabel('$t$')
plt.xscale('symlog')
plt.ylim([0, 1])
plt.ylabel('TMRCA CDF')
plt.tight_layout()
plt.show()